# Klassifikation

In diesem Notebook werden verschiedene Klassifikationsmodelle trainiert. Ziel ist es, die Städte im Datensatz aufgrund ihrer Feinstaubbelastung in zwei Klassen einzuteilen. Als Zielvariable wird also die Belastung mit mittelgroßen Feinstaubpartikeln (PM2.5, gemessen in µg/m³) angesetzt. 

Als Schwellenwert werden zwei Ansätze getestet:
1. WHO-Richtline von 5 µg/m³: fachlicher Standard
2. Median: datengetriebene Größe

Als Modelle werden verglichen:
1. Logistische Regression
2. Random Forest
3. Gradient Boosting

Verwendet werden verschiedene Module der Python Bibliothek **Scitkit-learn** für maschinelles Lernen

📌 **Datenstand:** `cleaned_air_quality_data_2025-03-27.csv`  
📁 **Importiert aus:** lokaler Datei (--> gitignore)

## 📚 Inhaltsverzeichnis 
(Diese Art von Inhaltsverzeichnis mit Link funktioniert leider in Notebooks nicht, weil die as JSON gespeichert werden und nicht als HTML...)

- [0. Datensatz laden](#0-datensatz-laden)
- [1. Dataframe vorbereiten](#1-dateframe-vorbereiten)
- [2. WHO-Richtlinie](#2-who-richtlinie)
- [3. Median der Zielvariablen](#3-median-der-zielvariablen)
- [4. Features, Target, Train-/Test-Split](#4-features-target-train-test-split)
- [5. Logistic Regression](#5-logistic-regression)
- [6. Random Forest](#6-random-forest)
- [7. Gradient Boosting](#7-gradient-boosting)


# 0. Datensatz laden

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [ ]:
df = pd.read_csv("data/cleaned_air_quality_data_2025-03-27.csv")
df.head()

# 1. Dateframe vorbereiten

Als Target benutzen wir mittelgroße Feinstaubpartikel (PM2.5), als Features alle anderen Schadstoffe im Datensatz: CO, NO₂, SO₂, O₃ und (im allerersten Modell) PM10.

Weil das Imputieren von Werten für nicht vorhandenen Kategorien (Schadstoffen) für eine Stadt nicht mehr als "Raten" ist, werden nur Städte in die Analyse mit aufgenommen, für die Messwerte zu allen sechs Luftschadstoffen vorliegen. (--> Wiebke: Was ich meine, ist: Ich kann innerhalb einer Stadt Werte imputieren, aber es ist ziemlicher Quatsch, die Werte von einer Stadt auf die andere zu übertragen, also stadtübergreifend zu imputieren.)

Die Luftqualität kann auch auf dem arithmetischen Mittel berechnet werden. Dies hat den Vorteil, dass die natürliche Varianz besser abgebildet wird und den Nachteil, dass Machine-Learning-Modelle sich bei der Klassifikation schlechter abschneiden. Beide Varianten wurden komplett durchgerechnet. Da hier das Verhalten unterschiedlicher Klassifikationsmodelle gezeigt werden soll, wurde für die finale Analyse der Median gewählt.

In [ ]:
# Liste relevanter Schadstoffe
pollutants = ['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']

In [ ]:
# Mittelwerte pro Stadt berechnen (Index = City)
df_median = df.groupby('City')[pollutants].median()

In [ ]:
# Für jede Stadt: Wie viele Mittelwerte sind vorhanden?
df_median['Num_Valid_Pollutants'] = df_median[pollutants].notna().sum(axis=1)

# Übersicht: Wie viele Städte haben wie viele gültige Schadstoffwerte?
coverage_summary = df_median['Num_Valid_Pollutants'].value_counts().sort_index()

# Ergebnis anzeigen
coverage_summary

In [ ]:
# Nur Städte mit allen 6 Schadstoff-Mittelwerten

df_median_complete = df_median[df_median['Num_Valid_Pollutants'] == 6]

len(df_median_complete)

Der Dataframe enthält nun 406 Städte, für die jeweils Werte für jeden Schadstoff vorliegen.

# 2. WHO-Richtlinie

Als erstes wird die Klasseneinteilung auf der Grundlage einen fachlichen Standards, nämlich des aktuellen WHO-Grenzwerts für PM2.5 von 5 µg/m³ vorgenommen:

In [ ]:
# Anzahl Städte mit guter/schlechter Luft (nach WHO-Grenzwert)
(df_median_complete['Pm25'] <= 5).value_counts()

In [ ]:
# Städte mit PM2.5 ≤ 5 µg/m³ filtern
clean_cities = df_median_complete[df_median_complete['Pm25'] <= 5]

# Ergebnis anzeigen
clean_cities

In [ ]:
# Anzahl gültiger PM2.5-Werte pro Stadt
pm25_counts = df.groupby('City')['Pm25'].count().sort_values()

# Zeige nur die "sauberen" Städte
pm25_counts.loc[['Plovdiv', 'Yazd']]

Dem WHO-Grenzwert entsprechen in unserem Datensatz nur zwei von 406 Städten: Plovdiv (Bulgarien), Yazd (Iran). Bei genauerem Hinschauen fällt allerdings auf, dass es für Plovidiv nur einen einzigen Messwert gibt und für Yazd nur sehr wenige. Die Messwerte sind damit nicht aussagekräftig.

Die WHO-Richtline kann also für das Training von Klassifikationsmodellen nicht als Schwellenwert verwendet werden - es hätte keine zweite Klasse, von der es lernen könnte.

**OFFENE FRAGE**: Ist es gut, für die restliche Analyse alle Städte drinzulassen, auch wenn sie nur wenige Messwerte pro Schadstoff haben? Sollte man da noch was aussortieren? Oder einfach mal so lassen, weil in der Realität die Datenqualität auch nur selten optimal ist?

# 3. Median der Zielvariablen

Als Alternative wird der Median als datengetriebener Grenzwert gewählt. Die Einteilung in Städte mit guter und schlechter Luftqualität basiert also auf dem Medianwert der durchschnittlichen PM2.5-Konzentration aller Städte im Datensatz. 

Durch die Verwendung des Medians entsteht eine ausgewogene Verteilung zwischen den beiden Klassen, die ein stabiles Training und eine faire Bewertung des Modells ermöglicht.

Um ein Klassifikationsmodell zur Vorhersage der Luftqualität von Städten zu erstellen, wird eine Zielvariable mit dem Namen **AirQualityLabel** eingeführt. Diese ordnet jeder Stadt eine von zwei Klassen zu:

- 0 → Gute Luftqualität

- 1 → Schlechte Luftqualität

Die Luftqualität wird aus den Medianen der einzelnen Schadstoffe pro Stadt berechnet. Städte mit einem PM2.5-Mittelwert über dem Median werden als "schlechte Luftqualität" (1) klassifiziert, alle anderen als "gute Luftqualität" (0).

In [ ]:
df_classification = df_median_complete.copy()

# Median von PM2.5 berechnen
pm25_median = df_classification['Pm25'].median()

# Zielvariable hinzufügen
df_classification['AirQualityLabel'] = (df_classification['Pm25'] > pm25_median).astype(int)

In [ ]:
# Umgang mit NaN-Werten

# Anzahl fehlender Werte pro Spalte
df_classification[pollutants].isna().sum()

In [ ]:
df_classification.sample(10)

In [ ]:
df_classification.shape

# 4. Features, Target, Train-/Test-Split

Der Datensatz für die Klassifikation (df_classification) enthält 406 Städte mit Medianwerten für die Schadstoffe, die als Features verwendet werden: CO, NO₂, SO₂, O₃ und (im allerersten Modell) PM10.

Als Target wird AirQualityLabel angesetzt, dessen Wert entsprechend dem Median von PM2.5 berechnet wurde (s. Abschnitt 3).

Wir verwenden den Standard-Trainings-Test-Split von scikit-learn und nehmen 80% der Daten für das Training und 20% für den Test.


In [ ]:
features_lr_pm10 = ['Co', 'No2', 'O3', 'Pm10', 'So2']
X = df_classification[features_lr_pm10]
y = df_classification['AirQualityLabel']

In [ ]:


# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 5. Logistic Regression

Als erstes Modell wird die logistische Regression verwendet. Die logistische Regression ist ein statistisches Modell zur Vorhersage binärer Ergebnisse, indem sie die Wahrscheinlichkeit eines Ereignisses anhand einer linearen Kombination von Variablen schätzt. Dabei wird die lineare Ausgabe über eine logistische Funktion in einen Wahrscheinlichkeitswert umgewandelt. Ein großer Vorteil ist die einfache Interpretierbarkeit und schnelle Berechnung, was sie zu einem guten Ausgangspunkt für Klassifikationsaufgaben macht. Allerdings eignet sie sich primär für lineare Zusammenhänge und stößt bei komplexeren Datenstrukturen schnell an ihre Grenzen.

Im ersten Modell wird PM10 als zusätzliches Feature eingebunden, um experimentell zu untersuchen, wie sich dieses stark mit PM2.5 korrelierte Merkmal auf das Modell auswirkt. Da PM2.5 als Basis zur Berechnung von AirQualityLabel dient, könnte PM10 das Modell verzerren, indem es indirekt bereits bekannte Informationen liefert. In der zweiten Runde wird PM10 entfernt, um zu evaluieren, wie sich die Modellleistung ohne diesen „künstlichen“ Informationsvorsprung verändert.

In [ ]:
#  Modelltraining
model_lr_pm10 = LogisticRegression(max_iter=1000)
model_lr_pm10.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred_lr_pm10 = model_lr_pm10.predict(X_test)
print(classification_report(y_test, y_pred_lr_pm10))

Mit PM10 als Feature liegt der F1-Score bei 90%.

In [ ]:
cm_lr_pm10 = confusion_matrix(y_test, y_pred_lr_pm10)

cm_df = pd.DataFrame(cm_lr_pm10, 
                     index=["Tatsächlich Negativ", "Tatsächlich Positiv"], 
                     columns=["Vorhergesagt Negativ", "Vorhergesagt Positiv"])

print(cm_df)


Das folgende Modell ist eien Wiederholung des vorhergehenden, aber ohne PM10 als Feature.

In [ ]:
features_lr = ['Co', 'No2', 'O3', 'So2']
X = df_classification[features_lr]
y = df_classification['AirQualityLabel']

In [ ]:
# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelltraining
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred_lr = model.predict(X_test)
print(classification_report(y_test, y_pred_lr))


Ohne PM10 als Feature wird nur noch ein F1-Score von 77% erreicht.

In [ ]:
cm_lr = confusion_matrix(y_test, y_pred_lr)

cm_df = pd.DataFrame(cm_lr, 
                     index=["Tatsächlich Negativ", "Tatsächlich Positiv"], 
                     columns=["Vorhergesagt Negativ", "Vorhergesagt Positiv"])

print(cm_df)


Beide Modelle erscheinen ausgewogen - keine der beiden Klassen wird bevorzugt. Das Modell mit PM10 als Feature erreicht mit 90% einen deutlich höheren F1-Score als das Modell ohne PM10 (F1: 77%).

Dennoch spricht einiges dafür, PM10 als Feature wegzulassen: Da PM10 hoch mit PM2.5 korreliert – aus dem ja das AirQualityLabel abgeleitet wird – besteht die Gefahr eines Informationslecks, bei dem das Modell indirekt schon „vorab“ Informationen über das Ziel erhält. Dies kann zu verzerrten Validierungsergebnissen und schlechterer Generalisierbarkeit führen, insbesondere wenn das Modell auf neuen, unbekannten Daten angewendet wird. Zudem erhöht die Einbeziehung hoch korrelierter Features das Risiko von Overfitting, wodurch das Modell zu sehr auf spezifische Trainingsdaten abgestimmt wird.

Daher ist es sinnvoll, PM10 zu entfernen, um ein robusteres und interpretierbareres Klassifikationsmodell zu entwickeln. Die folgenden Modelle werden nur mit dem reduzierten Feature-Satz berechnet.

# 6. Random Forest

Der Random Forest ist ein Ensemble-Verfahren, das viele Entscheidungsbäume (decision trees) kombiniert, um robustere und stabilere Vorhersagen zu erzielen. Jeder Baum wird auf einer zufälligen Teilmenge der Daten sowie einer zufälligen Auswahl von Merkmalen trainiert, wodurch das Modell insgesamt weniger anfällig für Überanpassung ist. Ein wesentlicher Vorteil ist seine hohe Genauigkeit und Robustheit gegenüber Ausreißern. Auf der anderen Seite leidet die Interpretierbarkeit, da es schwierig ist, den Einfluss einzelner Variablen im Ensemble nachzuvollziehen, und der Rechenaufwand kann bei großen Datensätzen erheblich sein.

In [ ]:
# Features and train test split repeated from previous model, just to make clear what is being used:

features_rf = ['Co', 'No2', 'O3', 'So2']
X = df_classification[features_rf]
y = df_classification['AirQualityLabel']

# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Random Forest-Modell erstellen
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Modell trainieren
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)


In [ ]:
# Vorhersagen & Bewertung drucken
print(classification_report(y_test, y_pred_rf))

In [ ]:
cm_rf = confusion_matrix(y_test, y_pred_rf)

cm_df = pd.DataFrame(cm_rf, 
                     index=["Tatsächlich Negativ", "Tatsächlich Positiv"], 
                     columns=["Vorhergesagt Negativ", "Vorhergesagt Positiv"])

print(cm_df)

Der Random Forest erreicht mit 82% einen leicht höheren F1-Score als die Logistische Regression. Auch diese Modell erscheint einigermaßen ausgewogen (6 false negatives zu 9 false positives).

Ein Nachteil des random Forests gegenüber der Logistischen Regression ist, dass der Einfluss einzelner Features auf das Modell im Ensemble schwer nachzuvollziehen ist. Hilfreiche Anhaltspunkte kann hier die Berechnung der Feature Importances geben, die zeigen, wie stark einzelne Features zur Verringerung der Unreinheit in den Entscheidungsbäumen beitragen. Allerdings sollte man sich bewusst sein, dass die Feature Importances nicht alle komplexen Wechselwirkungen zwischen den Features vollständig abbilden.

In [ ]:
# Feature importance extrahieren
importances = rf_model.feature_importances_

In [ ]:
# Feature Importance in ein DataFrame umwandeln
feature_importance_df = pd.DataFrame({
    'Feature': features_rf,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Feature Importance anzeigen mit "Feature" als Hue und deaktivierter Legende
plt.figure(figsize=(8, 4))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df,
            hue='Feature', palette='viridis', dodge=False)
plt.legend([], [], frameon=False)
plt.title('Feature Importance (Random Forest)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout();

plt.savefig("./images/Feature_Importance_Random_Forest.png", dpi=300, bbox_inches="tight")

Am wichtigsten für die Klassifizierung ist das Feature "CO". Welche Städte wurden falsch klassifiziert?

In [ ]:
# Falsch klassifizierte Städte finden
incorrect_predictions = X_test.copy()
incorrect_predictions['True Label'] = y_test
incorrect_predictions['Predicted Label'] = y_pred_rf

# Nur falsch klassifizierte Städte herausfiltern
incorrect_predictions = incorrect_predictions[incorrect_predictions['True Label'] != incorrect_predictions['Predicted Label']]

# CO-Werte der falsch klassifizierten Städte
incorrect_predictions['CO'] = X_test.loc[incorrect_predictions.index, 'Co']

# Ausgabe der Städte mit ihren CO-Werten
# incorrect_predictions[['True Label', 'Predicted Label', 'CO', 'So2', 'O3', 'No2']].sort_values(by='CO', ascending=False)
incorrect_predictions


In [ ]:
incorrect_predictions.index

Welche Messwerte (Median) haben die Städte im einzelnen?

In [ ]:
df_classification.loc[['Budapest', 'Okayama', 'Prato', 'Lampang', 'Nara-shi', 'Tuzla',
          'Abu dhabi', 'Worcester', 'Suncheon', 'San luis potosí', 'Zabrze',
          'Concepción', 'Tainan', 'Douliu', 'Winnipeg']]

In [ ]:
# CO-Werte der falsch klassifizierten Städte
co_values = incorrect_predictions['CO']

# Berechne die wichtigsten Statistiken (Durchschnitt, IQR)
co_mean = co_values.mean()
co_std = co_values.std()
co_min = co_values.min()
co_max = co_values.max()

# Berechne Interquartilsabstand (IQR)
Q1 = co_values.quantile(0.25)
Q3 = co_values.quantile(0.75)
IQR = Q3 - Q1

# Anzeigen der CO-Statistiken
print(f"Durchschnittlicher CO-Wert: {co_mean:.2f}")
print(f"Standardabweichung: {co_std:.2f}")
print(f"Minimaler CO-Wert: {co_min:.2f}")
print(f"Maximaler CO-Wert: {co_max:.2f}")
print(f"Interquartilsabstand (IQR): {IQR:.2f}")


Man sieht, dass das Modell vor allem dann falsch klassifiziert, wenn das Feature mit der höchsten Feature Importance (CO) unerwartet niedrig ist.

Ich glaube nicht, dass man da noch viel drehen kann. Das Modell ist eigentlich recht gut, wenn man bedenkt, dass die Messwerte zum Teil recht abenteuerlich sind.

# 7. Gradient Boosting

Gradient Boosting ist ein sequentielles Ensemble-Verfahren, bei dem schwache Lernmodelle – meist kleine Entscheidungsbäume – iterativ trainiert werden, um die Fehler der vorherigen Modelle zu korrigieren. Das Modell optimiert dabei schrittweise eine Verlustfunktion mithilfe von Gradientenabstieg, was zu einer sehr präzisen Vorhersage führt.

Zu den Vorteilen zählen eine hohe Leistungsfähigkeit und die Fähigkeit, komplexe Zusammenhänge zu erfassen. Allerdings kann Gradient Boosting bei unzureichender Abstimmung der Hyperparameter schnell überanpassen und ist in der Regel rechenintensiver als einfachere Modelle.

In [ ]:
# Gradient Boosting Modell erstellen
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Modell trainieren
gb_model.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred_gb = gb_model.predict(X_test)
print(classification_report(y_test, y_pred_gb))
# print(confusion_matrix(y_test, y_pred_gb))


In [ ]:
cm_gb = confusion_matrix(y_test, y_pred_gb)

cm_df = pd.DataFrame(cm_gb, 
                     index=["Tatsächlich Negativ", "Tatsächlich Positiv"], 
                     columns=["Vorhergesagt Negativ", "Vorhergesagt Positiv"])

print(cm_df)

Gradient Boosting erreicht mit 84% noch einmal einen leicht höheren F1-Score. Auch diese Modell erscheint einigermaßen ausgewogen (5 false negatives zu 8 false positives).